# SORT

1. 영상을 입력 받음<br>
YOLO 같은 "객체 탐지기"가 이미지에서 물체를 인식해 바운딩 박스를 생성<br>
2. 칼만 필터 예측<br>
칼만필터 = 시간에 따라 변하는 상태를 추정하는 방법, 모델의 관측값을 바탕으로 현재 상태를 재귀적으로 예측&업데이트하는 알고리즘<br>
3. 데이터 연결<br>
새로 탐지된 물체와 이전 프레임에서 추적한 물체를 매칭<br>
일치, 새로운 물체, 사라진 물체 로 구분<br>
4. 트랙 관리<br>
일치 - 업데이트 / 새로 나타난 물체 - 새 트랙에 등록 / 사라진 물체 - 일정시간 후 삭제<br>
5. 출력<br>
각 물체의 위치(바운딩 박스)와 고유한 ID를 출력
같은 물체는 같은 ID를 유지하여 "추적"

# DeepSort

1. 영상 입력을 받음<br>
위의 1번과 동일<br>
2. CNN 특징 추출<br>
탐지된 이미지를 Crop하여 CNN을 통해 각 물체의 고유한 특징 벡터(Appearance) 추출<br>
벡터 = 얼굴인식정보 의 역할<br>
3. 칼만필터 예측 - 이하 동일<br>
4. 데이터 연결<br>
칼만필터 예측 위치정보, YOLO 위치, CNN 특징 정보 조합하여 매칭<br>
5. 트랙 관리<br>
일정 시간동안 보이면 "확정 트랙(Confirmed Track)"으로 승격, 사라지면 삭제<br>
6. 출력<br>

1차 부감 영상의 경우 전혀 자동차를 잡아내지 못함.
YOLO가 상공에서 본 차량은 잘 인식하지 못한다는 gpt의 분석, 비교적 정면 화면으로 재시도

2차 도로cctv 영상은 전체적으로 잘 잡음

In [23]:
import cv2
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort

###########################
# 1. YOLO 11 모델 및 DeepSORT 초기화
yolo_model = YOLO('yolo11n.pt')  # YOLO 11 Nano 모델 (공식 가중치 나오는대로 파일명 확인 필요)
tracker = DeepSort(max_age=30)    # DeepSORT 초기화
###########################

In [25]:
# 2. 영상 열기
video_path = 'Traffic Camera.mp4'
cap = cv2.VideoCapture(video_path)

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
print(f"영상 정보: {width}x{height}, FPS: {fps}")
###########################

영상 정보: 1920x1080, FPS: 25.0


In [26]:
###########################
# 3. 결과 저장용 VideoWriter 설정
output_path = 'Parking_tracked02.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # mp4 코덱 설정
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))  # 출력 파일 설정
###########################

In [27]:
# 4. 프레임별 탐지 및 추적 (차량만 추적)
def detect_and_track(frame):
    results = yolo_model(frame)[0]  # YOLO 11 탐지 실행
    detections = []

    # COCO 기준 차량/이동수단 클래스 ID 리스트
    vehicle_classes = [2, 3, 5, 7]  # 자동차, 오토바이, 버스, 트럭

    for result in results.boxes:
        x1, y1, x2, y2 = map(int, result.xyxy[0])  # 바운딩 박스 좌표
        conf = result.conf[0].item()               # 신뢰도
        class_id = int(result.cls[0].item())       # 클래스 ID

        # 차량만 추적 (클래스 필터링)
        if class_id in vehicle_classes:
            w, h = x2 - x1, y2 - y1
            detections.append([[x1, y1, w, h], conf])  # <- DeepSORT 기대 형식

    # DeepSORT 업데이트 (추적 진행)
    tracks = tracker.update_tracks(detections, frame=frame)

    # 추적 결과 바운딩 박스와 ID 표시
    for track in tracks:
        if not track.is_confirmed():
            continue

        track_id = track.track_id
        ltrb = track.to_ltrb()  # (left, top, right, bottom)

        # 박스 그리기 & ID 표시
        cv2.rectangle(frame, (int(ltrb[0]), int(ltrb[1])), (int(ltrb[2]), int(ltrb[3])), (0, 255, 0), 2)
        cv2.putText(frame, f"ID: {track_id}", (int(ltrb[0]), int(ltrb[1]) - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    return frame

In [28]:
###########################

# 5. 메인 루프 - 영상 저장만 하고 화면 출력 안함
frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = detect_and_track(frame)
    out.write(frame)  # 처리된 프레임 저장

    frame_count += 1
    if frame_count % 50 == 0:  # 50프레임마다 진행상황 출력
        print(f"{frame_count} frames processed...")

cap.release()
out.release()
cv2.destroyAllWindows()

print(f"처리 완료. 저장 파일: {output_path}")
###########################


0: 384x640 2 cars, 1 truck, 1 traffic light, 23.1ms
Speed: 2.5ms preprocess, 23.1ms inference, 2.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 truck, 1 traffic light, 19.3ms
Speed: 3.9ms preprocess, 19.3ms inference, 5.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 2 cars, 1 traffic light, 10.9ms
Speed: 2.4ms preprocess, 10.9ms inference, 3.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 2 cars, 1 traffic light, 21.4ms
Speed: 2.5ms preprocess, 21.4ms inference, 4.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 2 cars, 1 truck, 1 traffic light, 20.0ms
Speed: 2.5ms preprocess, 20.0ms inference, 4.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 2 cars, 1 traffic light, 17.2ms
Speed: 2.5ms preprocess, 17.2ms inference, 3.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 2 cars, 1 truck, 1 traffic light, 12.2ms
Speed: 2.4ms preprocess